<a href="https://colab.research.google.com/github/timothyow/data_agent/blob/main/Data_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install langchain_community
!pip install langchain_openai

In [1]:
#read dataset
import pandas as pd

customers_df = pd.read_csv('/content/users_data.csv')
cards_df = pd.read_csv('/content/cards_data.csv')
transactions_df = pd.read_csv('/content/transactions_data.csv', sep=',', on_bad_lines='skip', dtype={'id': str, 'client_id': str, 'card_id': str, 'mcc': str, 'errors': str, 'zip': str})

# print(customers_df.head())
# print(cards_df.head())
# print(transactions_df.head())


/tmp/ipython-input-3354870882.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions_df = pd.read_csv('/content/transactions_data.csv', sep=',', on_bad_lines='skip', dtype={'id': str, 'client_id': str, 'card_id': str, 'mcc': str, 'errors': str, 'zip': str})


In [2]:
from IPython.display import display

display(customers_df.head())
display(cards_df.head())
display(transactions_df.head())

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935.0,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570.0,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092.0,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092.0,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051.0,Harwood,MD,20776.0,5813,NaN


data preprocessing
1. convert amount type fields to float
2. rename columns
3. normalize data

In [3]:
def clean_currency_columns(df: pd.DataFrame, currency_cols):
    """
    Remove $ and commas from given currency columns and convert to float.
    Also rename columns to add '_usd' suffix.
    """
    df = df.copy()
    for col in currency_cols:
        # Remove $ and commas, convert to float
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(r"[\$,]", "", regex=True)
            .astype(float)
        )
        # Rename column with _usd suffix
        df.rename(columns={col: f"{col}_usd"}, inplace=True)
    return df

In [4]:
currency_columns = ["per_capita_income", "yearly_income", "total_debt"]
customers_df_clean = clean_currency_columns(customers_df, currency_columns)

currency_columns = ["credit_limit"]
cards_df_clean = clean_currency_columns(cards_df, currency_columns)

In [5]:
# Let's rename the confusing columns
customers_df_clean.rename(columns={"id": "customer_id"}, inplace=True)
cards_df_clean.rename(columns={"id": "account_id", "client_id": "customer_id"}, inplace=True)

from IPython.display import display

print("Customers:")
display(customers_df_clean.head())

print("Cards:")
display(cards_df_clean.head())

Customers:


,customer_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income_usd,yearly_income_usd,total_debt_usd,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,37891.0,77254.0,191349.0,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,22681.0,33483.0,196.0,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,53797.0,109687.0,183855.0,675,1


Cards:


,account_id,customer_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit_usd,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,24295.0,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,21968.0,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,46414.0,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,12400.0,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,28.0,09/2008,2009,No


In [6]:
#normalize date: convert MM/YYYY to YYYY/MM-01
for col in ["expires", "acct_open_date"]:
  cards_df_clean[col] = pd.to_datetime(cards_df_clean[col], format="%m/%Y").dt.strftime("%Y-%m-01")


In [8]:
print(cards_df_clean.head())

   account_id  customer_id  card_brand        card_type       card_number  \
0        4524          825        Visa            Debit  4344676511950444   
1        2731          825        Visa            Debit  4956965974959986   
2        3701          825        Visa            Debit  4582313478255491   
3          42          825        Visa           Credit  4879494103069057   
4        4659          825  Mastercard  Debit (Prepaid)  5722874738736011   

      expires  cvv has_chip  num_cards_issued  credit_limit_usd  \
0  2022-12-01  623      YES                 2           24295.0   
1  2020-12-01  393      YES                 2           21968.0   
2  2024-02-01  719      YES                 2           46414.0   
3  2024-08-01  693       NO                 1           12400.0   
4  2009-03-01   75      YES                 1              28.0   

  acct_open_date  year_pin_last_changed card_on_dark_web  
0     2002-09-01                   2008               No  
1     2014-04-01

In [8]:
import os

# Create the directory for the database if it doesn't exist
database_dir = '/content/drive/MyDrive/database/'
if not os.path.exists(database_dir):
    os.makedirs(database_dir)

In [ ]:
# Load your DataFrames into your SQLite database using SQLAlchemy

import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite:///drive/MyDrive/database/mydb.db') # creating a connection object to a SQLite database file using SQLAlchemy.

# Load customer as a SQL table
customers_df_clean.to_sql('customers', engine, if_exists='replace', index=False)
del customers_df_clean

# Load cards as a SQL table
cards_df_clean.to_sql('cards', engine, if_exists='replace', index=False)
del cards_df_clean

# Load transactions as a SQL table
transactions_df.to_sql('transactions', engine, if_exists='replace', index=False)
del transactions_df



In [ ]:
# Read transaction data into chunks of 50000 and perform prepocessing. Total records in this dataset is 13,305,915
currency_columns = ["amount"]
chunksize = 50000

csv_path = '/content/transactions_data.csv'
for chunk in pd.read_csv(csv_path, chunksize=chunksize):
    chunk = clean_currency_columns(chunk, currency_columns)
    chunk.to_sql('transactions', engine, if_exists='append', index=False)

# Verify the tables by printing 5 rows from each
with engine.begin() as conn:
    print("Customers (from DB):")
    display(pd.read_sql('SELECT * FROM customers LIMIT 5', conn))

    print("Transactions (from DB):")
    display(pd.read_sql('SELECT * FROM transactions LIMIT 5', conn))

    print("Cards (from DB):")
    display(pd.read_sql('SELECT * FROM cards LIMIT 5', conn))

In [ ]:
# Verify the tables by printing 5 rows from each
with engine.begin() as conn:
    print("Customers rows (from DB):")
    display(pd.read_sql('SELECT count(*) FROM customers', conn))

    print("Transactions rows (from DB):")
    display(pd.read_sql('SELECT count(*) FROM transactions', conn))

    print("Cards rows (from DB):")
    display(pd.read_sql('SELECT count(*) FROM cards', conn))

In [ ]:
# Finally read json file into merchant_category table
import json

with open('/content/drive/MyDrive/database/mcc_codes.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(list(data.items()), columns=['merchant_category_code', 'merchant_description'])
df.to_sql('merchant_category', engine, if_exists='replace', index=False)
print("Merchant Category Information (from DB):")
with engine.begin() as conn:
    display(pd.read_sql('SELECT count(*) FROM merchant_category LIMIT 5', conn))

Configure LangChain wiht GPT, custom prompts and implement conversational memory.
This step sets up the core components of our advanced database agent:
LangChain for orchestration, writing custom prompts, using GPT for NLP and conversational memory for context retention.

In [9]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Enter API key")

Enter API key··········


In [12]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///drive/MyDrive/database/mydb.db")  # Update path if needed

# Wrap the SQLAlchemy engine in a LangChain SQLDatabase object
# This allows LangChain's SQL agent tools to interact with the database.
db = SQLDatabase(engine)

# Print a list of table names in the database that the agent is allowed to query
print(db.get_usable_table_names())

# Print schema details (column names, types, and constraints) for all usable tables
# This is useful for debugging or letting the LLM understand the database structure.
print(db.get_table_info())

['cards', 'customers', 'transactions']

CREATE TABLE cards (
	account_id BIGINT, 
	customer_id BIGINT, 
	card_brand TEXT, 
	card_type TEXT, 
	card_number BIGINT, 
	expires TEXT, 
	cvv BIGINT, 
	has_chip TEXT, 
	num_cards_issued BIGINT, 
	credit_limit_usd FLOAT, 
	acct_open_date TEXT, 
	year_pin_last_changed BIGINT, 
	card_on_dark_web TEXT
)

/*
3 rows from cards table:
account_id	customer_id	card_brand	card_type	card_number	expires	cvv	has_chip	num_cards_issued	credit_limit_usd	acct_open_date	year_pin_last_changed	card_on_dark_web
4524	825	Visa	Debit	4344676511950444	2022-12-01	623	YES	2	24295.0	2002-09-01	2008	No
2731	825	Visa	Debit	4956965974959986	2020-12-01	393	YES	2	21968.0	2014-04-01	2014	No
3701	825	Visa	Debit	4582313478255491	2024-02-01	719	YES	2	46414.0	2003-07-01	2004	No
*/


CREATE TABLE customers (
	customer_id BIGINT, 
	current_age BIGINT, 
	retirement_age BIGINT, 
	birth_year BIGINT, 
	birth_month BIGINT, 
	gender TEXT, 
	address TEXT, 
	latitude FLOAT, 
	longitude FLOAT,

In [15]:
# Import the OpenAI Chat model wrapper from LangChain for GPT models like gpt-4o, gpt-4
from langchain_openai import ChatOpenAI

# Construct a SQL agent from an LLM and toolkit or database.
from langchain_community.agent_toolkits.sql.base import create_sql_agent

# Constructs SQLDatabaseToolkit which will be passed to our agent
# This toolkit provides a set of tools for interacting with SQL databases (e.g., getting schema, running queries).
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

# Initialize the LLM (Language Model)
# model="gpt-4o" specifies the OpenAI model
# temperature=0 makes the model deterministic
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Create the SQL database toolkit
# db=db is the SQLDatabase instance we created earlier (wrapping the SQLite engine)
# llm=llm allows toolkit tools to use the LLM internally if needed
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Agent 1 - Uses default prompt
agent1 = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type="openai-tools" #specifies the agent framework for tool calling
)

In [18]:
query_nlq = "how many records are there in transactions table? And also return the final sql query"
response = agent1.invoke(query_nlq)
print(response["output"])

There are 0 records in the transactions table. 

The final SQL query used was:
```sql
SELECT COUNT(*) FROM transactions;
```


In [20]:
query_nlq = "which category has seen highest spending in 2015? And also return the final sql query"
response = agent1.invoke(query_nlq)
print(response["output"])

The query executed successfully, but it seems there was no data returned. This could be due to the absence of transactions in the year 2015 in the database. 

Here is the SQL query that was used:

```sql
SELECT mcc, SUM(CAST(amount AS FLOAT)) AS total_spending 
FROM transactions 
WHERE date LIKE '2015-%' 
GROUP BY mcc 
ORDER BY total_spending DESC 
LIMIT 1;
```

If you have any other questions or need further assistance, feel free to ask!


In [23]:
query_nlq = "how many accounts were opened in 2019? And also return the final sql query"
response = agent1.invoke(query_nlq)
print(response["output"])

A total of 88 accounts were opened in 2019. 

The SQL query used was:
```sql
SELECT COUNT(*) FROM cards WHERE acct_open_date LIKE '2019-%';
```


In [22]:
query_nlq = "Give me monthly distribution of accounts that were opened in 2019? return the sql query used to get the result"
response = agent1.invoke(query_nlq)
print(response["output"])

Here is the monthly distribution of accounts that were opened in 2019:

- January 2019: 9 accounts
- February 2019: 9 accounts
- March 2019: 13 accounts
- April 2019: 3 accounts
- May 2019: 2 accounts
- June 2019: 6 accounts
- July 2019: 9 accounts
- August 2019: 9 accounts
- September 2019: 7 accounts
- October 2019: 10 accounts
- November 2019: 5 accounts
- December 2019: 6 accounts

The SQL query used to obtain this result is:

```sql
SELECT strftime('%Y-%m', acct_open_date) AS month, COUNT(*) AS num_accounts_opened 
FROM cards 
WHERE strftime('%Y', acct_open_date) = '2019' 
GROUP BY month 
ORDER BY month;
```


In [24]:
query_nlq = "what is the average spend amount for merchant category department stores for 2015 ? And also return the final sql query"
response = agent1.invoke(query_nlq)
print(response["output"])

It seems there are no transactions recorded for the merchant category code "5311" (department stores) in the year 2015, or the data might not be available in the database. Therefore, the average spend amount for department stores in 2015 cannot be determined from the current dataset.

Here is the SQL query that was used:

```sql
SELECT AVG(CAST(amount AS FLOAT)) AS average_spend 
FROM transactions 
WHERE mcc = '5311' AND date LIKE '2015%';
```


In [25]:
query_nlq = "what was the total spend for this year? And also return the final sql query"
response = agent1.invoke(query_nlq)
print(response["output"])

The total spend for this year is not available in the database, possibly due to missing or incomplete data for the current year. 

The SQL query used was:
```sql
SELECT SUM(CAST(amount AS REAL)) AS total_spend FROM transactions WHERE strftime('%Y', date) = strftime('%Y', 'now')
```


In [29]:
#print the default prompt
from langchain.agents.agent_toolkits.sql.prompt import SQL_PREFIX, SQL_SUFFIX
print(SQL_PREFIX)
print(SQL_SUFFIX)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I

top_k: The maximum number of results to return from a database query, used to limit output size.

agent_scratchpad: A field where the agent tracks its intermediate thoughts, steps, or tool invocations during reasoning.

dialect: The specific SQL language variant (such as SQLite, MySQL, or PostgreSQL) the agent should use when generating queries.

input: The user's natural language question or instruction provided to the agent.

### Let's add custom instruction to default prompt

In [34]:
# Let's add custom instruction to default prompt.

from langchain_core.prompts import PromptTemplate
from datetime import date
custom_prompt = PromptTemplate(
    input_variables=["current_date"],
    template="""
You are an agent designed to interact with a SQL database. Current date is {current_date}.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Begin!

Question: {input}
Thought: I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables.
{agent_scratchpad}
"""
)

current_date = str(date.today())
# Agent 2 - Uses custom prompt
agent2 = create_sql_agent(
    llm=llm,
    prompt=custom_prompt,
    toolkit=toolkit,
    agent_type="openai-tools",
)

In [38]:
#using agent 2
query_nlq = "what was the total spend for this year? And also return the final sql query"
response = agent2.invoke({"input": query_nlq, "current_date": current_date})
print(response["output"])

The total spend for this year (2025) is not available in the database, as the query returned `None`. 

Here is the final SQL query that was used:

```sql
SELECT SUM(CAST(amount AS REAL)) AS total_spend FROM transactions WHERE date >= '2025-01-01' AND date <= '2025-12-31';
```


### Let's add conversational memory

In [39]:
from langchain.memory import ConversationBufferMemory

custom_prompt = PromptTemplate(
    input_variables=["chat_history", "current_date"],
    template="""
You are an agent designed to interact with a SQL database. Current date is {current_date}.
Given an input question and history of conversation, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.
If the question is ambiguous or appears like a follow-up question always refer to History of Conversation to get more context. Even after that if the query remains ambiguous, just return "please give more info"

Begin!

Question: {input}
Thought: I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables.
{agent_scratchpad}
History of Conversation
{chat_history}
"""
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="input")
agent3 = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    prompt=custom_prompt,
    agent_type="openai-tools",
    agent_executor_kwargs={
        "memory": memory
    }
)


/tmp/ipython-input-3294402511.py:29: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="input")


In [40]:
response = agent3.invoke({"input": "which category saw highest spend in 2016? And also return the final sql query", "current_date": current_date})
print(response["output"])

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-zqRk2vvHyBw9JRmwAnHp9YRv on tokens per min (TPM): Limit 30000, Used 26355, Requested 3982. Please try again in 674ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
query_nlq_followup = " now tell me the category for 2017. Also generate the final sql"

response_followup = agent3.invoke({"input": query_nlq_followup, "current_date": current_date})
print(response_followup["output"])